In [7]:
# importing libraries
import cv2
import math
import asyncio
import numpy as np
import skimage.io as io
from scipy.stats import norm

In [8]:
def gaussian(x, sigma):
    return (1.0 / math.sqrt(2 * math.pi) * sigma) * math.exp(-1 * (x ** 2) / (2 * sigma ** 2))

In [9]:
def bilateral_filter(source_img, start_raw, end_raw, start_col, end_col, sigma):
    filtered = np.copy(source_img)
    
    for r in range(start_raw, end_raw):
        for c in range(start_col, end_col):
            ks = 0
            w = 0
            for i in range(-1, 2):
                if i + c >= start_col and i + c < end_col:
                    # Target pixel
                    p = source_img[r][i + c]
                    # Current pixel
                    s = img[r][c]
                    weight = gaussian(abs(p - s), sigma)
                    ks += weight * p
                    w += weight
            filter1 = ks / w
            for i in range(-1, 2):
                if i + r >= start_raw and i + r < end_raw:
                    # Target pixel
                    p = source_img[i + r][c]
                    # Current pixel
                    s = img[r][c]
                    weight = gaussian(abs(p - s), sigma)
                    ks += weight * p
                    w += weight
            filter2 = ks / w
            filtered[r][c] = (filter1 + filter2) / 2
    return filtered

In [12]:
# get the histogram of the image
def get_hist(img):
    hist_img = np.zeros((256))
    for i in range(len(img)):
        for j in range(len(img[0])):
            hist_img[img[i][j]] +=1
    return hist_img

def find_median(histogram, aSize):
    histogram = np.cumsum(histogram)
    for i in range(len(histogram)):
        if histogram[i] > (aSize ** 2) / 2:
            return i
    return 0

def add_to_hist(histogram, arr):
    for i in arr:
        histogram[i] += 1

def sub_from_hist(histogram, arr):
    for i in arr:
        if(histogram[i]):
            histogram[i] -= 1

async def bilateral_filter_own(img, aSize):
    source_img = np.copy(img)
    raw, col = source_img.shape
    offset = aSize // 2
    hist_img = get_hist(img[0:aSize, 0:aSize])
    source_img[offset][offset] = find_median(hist_img, aSize)
    i = offset
    j = offset + 1
    while i in range(offset, raw - offset):
        while j in range(offset + 1, col - offset):
            add_to_hist(hist_img, img[i - offset: i + offset + 1, j + offset])
            sub_from_hist(hist_img, img[i - offset: i + offset + 1, j - offset])
            source_img[i][j] = find_median(hist_img, aSize)
            j += 1
        i += 1
        j = offset + 1
        hist_img = get_hist(img[i - offset: i + offset, 0 : aSize])
        source_img[i][j - 1] = find_median(hist_img, aSize)
    return source_img


In [18]:
# reading image
if __name__ == "__main__":
	img = cv2.imread("ross.jpg")
	img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

	'''
	cv2.imshow("ross", img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()
	'''

	print(img.shape)
	cartoon  = bilateral_filter_own(img[200: 500, 200: 400], 7)
	median = cv2.medianBlur(img, 7)
	#cartoon = cv2.bilateralFilter(img, 10, 250, 250)
	cv2.imshow("ross", cartoon)
	cv2.imshow("org ross", median)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

	'''
	# Edges
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	gray = cv2.medianBlur(gray, 5)
	edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
											cv2.THRESH_BINARY, 9, 9)

	# Cartoonization
	color = cv2.bilateralFilter(img, 15, 250, 250)
	cartoon = cv2.bitwise_and(color, color, mask=edges)


	cv2.imshow("Image", img)
	cv2.imshow("Cartoon", cartoon)
	'''


(688, 480)
